In [ ]:
import numpy as np
import pylab as plt
import drawSvg as draw
from skimage import io, filters, measure, feature, exposure, morphology
from scipy import ndimage
from scipy.ndimage import rotate
import os
import random
import inspect
%matplotlib inline

In [ ]:
toDraw = 'MobiusAlpha/'
ao_monkey = io.imread(toDraw + 'AO.png')
uv_monkey = io.imread(toDraw + 'UV.png')
normal_monkey = io.imread(toDraw + 'Normal.png')
original_monkey = io.imread(toDraw + 'Original.png')

fig=plt.figure(figsize=(20, 20))

fig.add_subplot(2, 2, 1)
plt.imshow(ao_monkey)
fig.add_subplot(2, 2, 2)
plt.imshow(uv_monkey)
fig.add_subplot(2, 2, 3)
plt.imshow(normal_monkey)
fig.add_subplot(2, 2, 4)
plt.imshow(original_monkey)
fig.tight_layout()

In [ ]:
edge_monkey = filters.sobel(normal_monkey[:,:,0]) + filters.sobel(ao_monkey[:,:,0])
#print(edge_monkey)
#edge_monkey = ndimage.gaussian_filter(edge_monkey, sigma=0.6)
plt.figure(figsize=(20,20))
plt.imshow((edge_monkey), cmap = plt.cm.Greys)

In [ ]:
new_edge_monkey =np.rot90(edge_monkey, k=-1)
contours = measure.find_contours(new_edge_monkey, 0.66, fully_connected="high")
simplified_contours = [measure.approximate_polygon(c, tolerance=0) for c in contours]
print(len(simplified_contours))
plt.figure(figsize=(6,6))
for n, contour in enumerate(simplified_contours):
    plt.plot(contour[:, 0], contour[:, 1], linewidth=1, color="#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)]))

In [ ]:
histogram = exposure.histogram(original_monkey[:,:,0])
plt.figure()
plt.hist(histogram[0], histogram[1])

In [ ]:
original_monkey_shade = morphology.erosion(np.copy(original_monkey)[:,:,0] - edge_monkey)
shade_monkey = np.where(original_monkey_shade < 24, 256, 0)
print(np.where(original_monkey_shade < 24))
plt.figure(figsize=(20,20))
plt.imshow(original_monkey[:,:,1]-np.zeros_like(edge_monkey), cmap=plt.cm.Greys)

In [ ]:
vector_shade_monkey = np.rot90(shade_monkey, k=-1)
contours = measure.find_contours(vector_shade_monkey, 0.2, fully_connected="high")
simplified_contours = [measure.approximate_polygon(c, tolerance=0) for c in contours]
plt.figure(figsize=(6,6))
plt.axis("off")
for n, contour in enumerate(simplified_contours):
    plt.plot(contour[:, 0], contour[:, 1], linewidth=0.5, color="black")
plt.savefig("suzzane.svg")

In [ ]:
original_shade = morphology.erosion(np.copy(original_monkey)[:,:,0] - edge_monkey)
shaded_area = np.where(original_shade >= 60)
shade = np.where(original_shade < 60, 0, 256)
ao_image = np.where(ao_monkey[:,:,0]>0)
output = ao_monkey[:,:,0]-shade
output_coords = np.where(output>0)
output = np.where(output > 0, 256, 0)
mask = np.zeros_like(edge_monkey)
changed_row = mask[0]
changed_row[::4] = 256
for i in range(0, len(mask)):
    mask[i] = changed_row
    changed_row = np.roll(changed_row, -1)
output = output+mask-256
output = np.where(output > 0, 256, 0)
print(output_coords)
plt.imshow(output, cmap=plt.cm.Greys, vmax = 256, vmin = 0) 

In [ ]:
original_shade = morphology.erosion(np.copy(original_monkey)[:,:,0] - edge_monkey)
shaded_area = np.where(original_shade >= 24)
original_shade[::-8] = 256
shade = np.where(original_shade < 24, 0, 256)
shade[shaded_area] = 0
plt.imshow(shade, cmap=plt.cm.Greys, vmax = 256, vmin = 0) 

In [ ]:
#fd, hog_image = feature.hog(original_monkey[:,:,1], block_norm='L2-Hys', visualise=True)
#hog_image = exposure.rescale_intensity(hog_image)
object_coords = np.where(uv_monkey[:,:,1]!=0)
print(object_coords)
band_coords = np.where(uv_monkey[:,:,1]==99, 256, 0)
print(band_coords)
print(uv_monkey[:,:,1][166,474])
plt.imshow(band_coords)